In [1]:
import requests, json, os, sys, time, re
from datetime import datetime
from multiprocessing import Pool
from sqlalchemy import *

In [4]:
def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = int(len(lst_long) / n)
    else:
        totalBatches = int(len(lst_long) / n) + 1
    for i in range(totalBatches):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted


def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 1. * currentCount / totalCount * 100
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\rStatus: [{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print ('\n')

In [10]:
path_user_id = 'data/steam_user_id.txt'
with open(path_user_id, 'rb') as f:
    lst_user_id = f.readlines()[:100]

In [14]:
lst_user_id[:5]

[b'76561198158086086\n',
 b'76561198074188133\n',
 b'76561198058088990\n',
 b'76561198175177483\n',
 b'76561198042649112\n']

In [17]:
for user_id in lst_user_id[:5]:
    base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
    params = {
        'key' : 'C509C52E59B304D96B874CA379C34ED7',
        'steamid' : user_id.strip(),
        'format' : 'json'         
    }
    r = requests.get(base_url, params = params)
    user_inventory = r.json().get('response').get('games')
    time.sleep(.5)
    print(user_id, '\n', user_inventory)

b'76561198158086086\n' 
 None
b'76561198074188133\n' 
 [{'appid': 4000, 'playtime_forever': 3415}, {'appid': 34030, 'playtime_forever': 16526}, {'appid': 42680, 'playtime_forever': 4631}, {'appid': 42690, 'playtime_forever': 11055}, {'appid': 207610, 'playtime_forever': 126}, {'appid': 50300, 'playtime_forever': 625}, {'appid': 104900, 'playtime_forever': 173}, {'appid': 227300, 'playtime_forever': 1447}, {'appid': 113400, 'playtime_forever': 1120}, {'appid': 203290, 'playtime_forever': 4008}, {'appid': 206210, 'playtime_forever': 114}, {'appid': 211500, 'playtime_forever': 11}, {'appid': 218230, 'playtime_forever': 327}, {'appid': 1083500, 'playtime_forever': 0}, {'appid': 236390, 'playtime_forever': 1055}, {'appid': 107400, 'playtime_forever': 0}, {'appid': 224260, 'playtime_forever': 1283}, {'appid': 65790, 'playtime_forever': 0}, {'appid': 233450, 'playtime_forever': 2841}, {'appid': 217200, 'playtime_forever': 5}, {'appid': 242760, 'playtime_forever': 2168}, {'appid': 107410, 'pla

b'76561198175177483\n' 
 None
b'76561198042649112\n' 
 [{'appid': 4000, 'playtime_forever': 1134}, {'appid': 220, 'playtime_forever': 131}, {'appid': 340, 'playtime_forever': 0}, {'appid': 400, 'playtime_forever': 0}, {'appid': 2700, 'playtime_forever': 988}, {'appid': 13520, 'playtime_forever': 0}, {'appid': 19900, 'playtime_forever': 0}, {'appid': 20510, 'playtime_forever': 25}, {'appid': 10090, 'playtime_forever': 828}, {'appid': 9480, 'playtime_forever': 0}, {'appid': 3590, 'playtime_forever': 174}, {'appid': 3700, 'playtime_forever': 0}, {'appid': 550, 'playtime_forever': 3241}, {'appid': 35140, 'playtime_forever': 66}, {'appid': 33220, 'playtime_forever': 643}, {'appid': 33900, 'playtime_forever': 6}, {'appid': 33930, 'playtime_forever': 154}, {'appid': 219540, 'playtime_forever': 1}, {'appid': 24780, 'playtime_forever': 0}, {'appid': 8980, 'playtime_forever': 53}, {'appid': 729040, 'playtime_forever': 0}, {'appid': 22380, 'playtime_forever': 57}, {'appid': 40800, 'playtime_forev

In [11]:
def worker(lst_user_id_temp):
    dic_temp = {}
    for user_id in lst_user_id_temp:
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
        params = {
            'key' : 'D0C62157A8941F12A687382B6D635449',
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        user_inventory = r.json().get('response').get('games')
        dic_temp.update({user_id.strip():user_inventory})
        time.sleep(.5)
    return dic_temp

In [12]:
len(lst_user_id)

100

In [45]:
p = Pool(2)

total_count = len(lst_user_id)
current_count = 0
show_work_status(0, total_count, current_count)

dic_master = {}
for i in split_list(lst_user_id,50):
    lst_temp_dic = p.map(worker, split_list(i,25))
    for j in lst_temp_dic:
        dic_master.update(j)
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
    time.sleep(5)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [46]:
i = 0
for keys in list(dic_master.keys()):
    print(keys)
    i +=1
    if i ==2:
        break

b'76561198158086086'
b'76561198074188133'


In [47]:
i = 0
for user_id, user_inventory in list(dic_master.items()):
    print(user_id, user_inventory)
    print(type(user_id))
    print(type(user_inventory))
    i +=1
    if i ==2:
        break

b'76561198158086086' None
<class 'bytes'>
<class 'NoneType'>
b'76561198074188133' [{'appid': 4000, 'playtime_forever': 3415}, {'appid': 34030, 'playtime_forever': 16526}, {'appid': 42680, 'playtime_forever': 4631}, {'appid': 42690, 'playtime_forever': 11055}, {'appid': 207610, 'playtime_forever': 126}, {'appid': 50300, 'playtime_forever': 625}, {'appid': 104900, 'playtime_forever': 173}, {'appid': 227300, 'playtime_forever': 1447}, {'appid': 113400, 'playtime_forever': 1120}, {'appid': 203290, 'playtime_forever': 4008}, {'appid': 206210, 'playtime_forever': 114}, {'appid': 211500, 'playtime_forever': 11}, {'appid': 218230, 'playtime_forever': 327}, {'appid': 1083500, 'playtime_forever': 0}, {'appid': 236390, 'playtime_forever': 1055}, {'appid': 107400, 'playtime_forever': 0}, {'appid': 224260, 'playtime_forever': 1283}, {'appid': 65790, 'playtime_forever': 0}, {'appid': 233450, 'playtime_forever': 2841}, {'appid': 217200, 'playtime_forever': 5}, {'appid': 242760, 'playtime_forever': 21

In [50]:
with open('data/crawled_user_inventory.txt', 'w') as f:
    for user_id, user_inventory in list(dic_master.items()):
        f.write(json.dumps({str(user_id):user_inventory}))
        f.write('\n')

In [51]:
r.headers

{'X-eresult': '1', 'Content-Type': 'application/json; charset=UTF-8', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '2153', 'Expires': 'Fri, 02 Aug 2019 00:04:24 GMT', 'Date': 'Fri, 02 Aug 2019 00:04:24 GMT', 'Connection': 'keep-alive'}

In [52]:
# steam app info

In [53]:
r = requests.get('http://steamspy.com/api.php?request=all')
dic_app_user = r.json()

In [57]:
r = requests.get('http://api.steampowered.com/ISteamApps/GetAppList/v2')
dic_app_list = r.json()
lst_app_id = [i.get('appid') for i in dic_app_list.get('applist').get('apps')]

In [58]:
len(lst_app_id)

82693

In [65]:
path_app_detail = 'data/app_detail.txt'

how to get data from webstie. Do not need if you already have app_detail.txt

In [64]:
lst_app_id = dic_app_user.keys()
total_count = len(lst_app_id)
current_count = 0
show_work_status(0, total_count, current_count)


with open(path_app_detail, 'w') as f:
    for app_id in lst_app_id:
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3): # 3 times to try
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except:
                time.sleep(5)
                pass
        f.write(json.dumps(result))
        f.write('\n')
        show_work_status(1, total_count, current_count)
        current_count += 1
        if current_count % 200 == 0:
            time.sleep(300)
        else:
            time.sleep(.5)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [67]:
with open(path_app_detail, 'r') as f:
    dic_steam_app = {'initial_price':{},'name':{},'score':{},'windows':{},'mac':{},'linux':{},'type':{},'release_date':{},'recommendation':{},'header_image':{}}
    lst_raw_string = f.readlines()[:3000]
    total_count = len(lst_raw_string)
    current_count = 0
    show_work_status(0, total_count, current_count)
    for raw_string in lst_raw_string:
        app_data = list(json.loads(raw_string).values())[0]
        if app_data != {}:
            steam_id = app_data.get('steam_appid')
            initial_price = app_data.get('price_overview',{}).get('initial')
            if app_data.get('is_free') == True:
                initial_price = 0
            app_name = app_data.get('name')
            critic_score = app_data.get('metacritic', {}).get('score')
            app_type = app_data.get('type')
            for (platform, is_supported) in app_data.get('platforms',{}).items():
                if is_supported == True:
                    dic_steam_app[platform].update({steam_id:1})
            if app_data.get('release_date',{}).get('coming_soon') == False:
                release_date = app_data.get('release_date',{}).get('date')
                if not release_date == '':
                    try:
                        release_date = datetime.strptime(release_date, '%b %d, %Y')
                    except:
                        try:
                            release_date = datetime.strptime(release_date, '%d %b, %Y')
                        except:
                            release_date = None
                    

            recommendation = app_data.get('recommendations',{}).get('total')
            header_image = app_data.get('header_image')
            dic_steam_app['initial_price'].update({steam_id:initial_price})
            dic_steam_app['name'].update({steam_id:app_name})
            dic_steam_app['score'].update({steam_id:critic_score})
            dic_steam_app['type'].update({steam_id:app_type})
            dic_steam_app['release_date'].update({steam_id:release_date})
            dic_steam_app['recommendation'].update({steam_id:recommendation})
            dic_steam_app['header_image'].update({steam_id:header_image})
        show_work_status(1, total_count, current_count)
        current_count += 1

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [68]:
import pandas as pd
df_app_info = pd.DataFrame(dic_steam_app)

In [69]:
df_app_info.head()

,initial_price,name,score,windows,mac,linux,type,release_date,recommendation,header_image
60,499,Ricochet,NaN,1,1.0,1.0,game,2000-11-01,2109.0,http://cdn.akamai.steamstatic.com/steam/apps/6...
300,999,Day of Defeat: Source,80.0,1,1.0,1.0,game,2010-07-12,7753.0,http://cdn.akamai.steamstatic.com/steam/apps/3...
550,1999,Left 4 Dead 2,89.0,1,1.0,1.0,game,2009-11-16,159088.0,http://cdn.akamai.steamstatic.com/steam/apps/5...
1300,999,SiN Episodes: Emergence,75.0,1,NaN,NaN,game,2006-05-10,408.0,http://cdn.akamai.steamstatic.com/steam/apps/1...
1640,649,Disciples II: Gallean's Return,84.0,1,NaN,NaN,game,2006-07-06,448.0,http://cdn.akamai.steamstatic.com/steam/apps/1...


In [75]:
engine = create_engine('mysql+pymysql://:@localhost/game_recommendation?charset=utf8mb4')
df_app_info.to_sql('tbl_app_info_test',engine, if_exists='replace')

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)") (Background on this error at: http://sqlalche.me/e/e3q8)

have troubles connect to mysql even when putting my user name and password. Any setting for mysql?